In [354]:
import numpy as np
import cplex
import random

In [355]:
class Subproblem_LP():
    def __init__(self):
        self.n_dim = (n_stage - 1) * n_job * n_job
        self.n_constraints = 2 * (n_stage - 1) * n_job
        self.variable_names = ['x'+ str(i) for i in range(self.n_dim)]
        self.constraints_rows = self.set_constraints_byrows()
        self.rhs = [1] * self.n_constraints
        self.senses = 'E'* self.n_constraints
        self.Subproblem_LP_TU = cplex.Cplex()
        self.opt_solution = []
        
    def set_job_to_position(self):
        job_to_position_onlyone = []
        for i in range((n_stage - 1) * n_job):
            job_to_position_onlyone.append([self.variable_names[i * n_job : (i + 1) * n_job], [1] * n_job])
        return job_to_position_onlyone
    
    def set_position_to_job(self):
        position_to_job_onlyone = []
        for j in range(n_stage - 1):
            for i in range(n_job):
                position_to_job_onlyone.append([self.variable_names[j * (n_job**2) + i: (j + 1) * (n_job**2): n_job], [1] * n_job])
        return position_to_job_onlyone
    
    def set_constraints_byrows(self):
        job_to_position_onlyone, position_to_job_onlyone = self.set_job_to_position(), self.set_position_to_job()
        return job_to_position_onlyone + position_to_job_onlyone
    
    def compute_obj(self):
        return [random.random() for i in range(self.n_dim)]
    
    def set_method(self, method):
        alg = self.Subproblem_LP_TU.parameters.lpmethod.values

        if method == "o":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.auto)
        elif method == "p":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.primal)
        elif method == "d":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.dual)
        elif method == "b":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.barrier)
            self.Subproblem_LP_TU.parameters.barrier.crossover.set(self.Subproblem_LP_TU.parameters.parameters.barrier.crossover.values.none)
        elif method == "h":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.barrier)
        elif method == "s":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.sifting)
        elif method == "c":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.concurrent)
        else:
            raise ValueError('method must be one of "o", "p", "d", "b", "h", "s" or "c"')
        
    def solve(self, method = 'o'):
        self.Subproblem_LP_TU.objective.set_sense(self.Subproblem_LP_TU.objective.sense.maximize)
        LP_TU_obj = self.compute_obj()
        ub = [1] * self.n_dim
    
        self.Subproblem_LP_TU.variables.add(obj = LP_TU_obj, ub = ub, names = self.variable_names)    
    
        assert(self.n_constraints == len(self.constraints_rows))
        row_names = ['c'+ str(i) for i in range(self.n_constraints)]
        print("the dimension of x:", self.n_dim, "the number of constraints:", self.n_constraints)
        self.Subproblem_LP_TU.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = row_names)
        self.set_method(method)
        self.Subproblem_LP_TU.solve()
        self.get_solvable_status()
        self.get_optvalues()
        
    def write_model(self):
        self.Subproblem_LP_TU.write('Subproblem_LP.lp')
        
    def get_optvalues(self):
        for i, x in enumerate(self.Subproblem_LP_TU.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.Subproblem_LP_TU.solution.get_status()
        if status == self.Subproblem_LP_TU.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.Subproblem_LP_TU.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.Subproblem_LP_TU.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def output_LP_result(self):
        with open('Subproblem_LP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
            f.write('\n')

In [356]:
class Subproblem_QP():
    def __init__(self):
        self.n_dim = n_job * n_stage + 2 * pcast * n_machine['steelmaking']
        self.variablenames = ['t' + str(i)  for i in range(self.n_dim)]
        self.Subproblem_QP = cplex.Cplex()
        self.duedate = self.compute_duedate()
        self.constraints_rows, self.rhs, self.n_per_constraints = self.set_constraints_byrows()
        self.senses = self.set_senses()
        self.obj_linear, self.qmat = self.compute_obj_linear(), self.compute_obj_qmat()
        self.opt_solution = []
        self.n_constraints = len(self.constraints_rows)
        
    def set_preceding_transfer_constraints(self):
        preceding_transfer_constraints = []
        for i in range(n_job * (n_stage - 1)):
            preceding_transfer_constraints.append([[self.variablenames[i], self.variablenames[i + n_job]], [1, -1]])

        rhs_transittime, rhs_processtime = [], processtime[0: n_job * (n_stage - 1)]
        for key in transittime:
            rhs_transittime += [transittime[key]] * n_job
        rhs = [ - processtime[i] - rhs_transittime[i] for i in range(n_job * (n_stage - 1))]
        return preceding_transfer_constraints, rhs, rhs_processtime, rhs_transittime

    def set_process_continuous_constraints(self):
        process_continuous, adjacent_casts = [], []
        rhs_continuous_processtime, rhs_adjacent_casts = [], []
        assert(n_job * (n_stage - 1) % n_batch == 0)
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if (i + 1) % n_batch != 0:
                process_continuous.append([[self.variablenames[i], self.variablenames[i + 1]], [-1, 1]])
                rhs_continuous_processtime.append(processtime[i])
            else:
                if (i + 1) % (n_batch * pcast) != 0:
                    adjacent_casts.append([[self.variablenames[i], self.variablenames[i + 1]], [-1, 1]])
                    rhs_adjacent_casts.append(- processtime[i] - su)
        return process_continuous, adjacent_casts, rhs_continuous_processtime, rhs_adjacent_casts 

    def set_additional_constraints(self):
        additional_constraints = []
        index = n_job * n_stage
        index_start = pcast * n_machine['casting']
        assert(n_job * (n_stage - 1) % n_batch == 0 and (index % n_batch == 0))
        batch = 0
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if i % n_batch == 0:
                additional_constraints.append([[self.variablenames[i], self.variablenames[index + batch], self.variablenames[index + batch + index_start]], [1, -1, 1]])
                batch += 1
        return additional_constraints
    
    def set_constraints_byrows(self):
        preceding_transfer_constraints, rhs_preceding_transfer, _, _ = self.set_preceding_transfer_constraints()
        process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts = self.set_process_continuous_constraints()
        additional_constraints = self.set_additional_constraints()
    
        rows = preceding_transfer_constraints + process_continuous_constraints + adjacent_casts_constraints + additional_constraints
        
        n_per_constraints = [len(preceding_transfer_constraints), len(process_continuous_constraints), len(adjacent_casts_constraints), len(additional_constraints)]
        
        rhs = rhs_preceding_transfer + rhs_continuous_processtime + rhs_adjacent_casts + self.duedate
        return rows, rhs, n_per_constraints
    
    def set_senses(self):
        s, senses = 'LELE', ''
        for i in range(len(self.n_per_constraints)):
            senses += s[i] * self.n_per_constraints[i]
        return senses
    
    def compute_duedate(self):
        duedate = [0 for i in range(pcast * n_machine['casting'])]
        duedate_start = [0 for i in range(n_machine['casting'])]
        n_cast = pcast * n_machine['casting']
        for i in range(n_machine['casting']):
            index = 0
            for key in transittime:
                duedate_start[i] += processtime[i * n_cast + index * n_job] + transittime[key]
                index += 1
        print(duedate_start)
    
        for i in range(n_machine['casting']):
            duedate[i * pcast] = duedate_start[i]
            index_start = n_job * (n_stage - 1) + i * n_cast
            for j in range(1, pcast):
                duedate[i * pcast + j] = duedate[i * pcast + j - 1] + sum(processtime[index_start: index_start + n_batch]) + su
                index_start = index_start + n_batch
        return duedate
    
    def compute_obj_wait_time(self):
        obj_wait_time = np.zeros((n_stage, n_job))
        for i in range(n_stage - 1):
            for j in range(n_job):
                obj_wait_time[i + 1][j], obj_wait_time[i][j] = obj_wait_time[i + 1][j] + coeff[i], obj_wait_time[i][j] - coeff[i]
        obj_wait_time = obj_wait_time.reshape(1, n_stage * n_job)
        return np.hstack((obj_wait_time, np.zeros((1, n_dim - n_stage * n_job))))

    def compute_obj_casting(self):
        obj_casting = np.hstack((d_coeff['d1'] *  np.ones((1, pcast * n_machine['casting'])), d_coeff['d2'] * np.ones((1, pcast * n_machine['casting']))))
        obj_casting = np.hstack((np.zeros((1, n_job * n_stage)), obj_casting))
        return obj_casting
        
    def compute_obj_linear(self):
        return [1.0 for i in range(self.n_dim)]
    
    def compute_obj_qmat(self):
        qmat = []
        for i in range(self.n_dim):
            qmat.append([self.variablenames, [- 1 for i in range(self.n_dim)]])
        return qmat
    
    def solve(self):
        self.Subproblem_QP.objective.set_sense(self.Subproblem_QP.objective.sense.maximize)
        
        ub   = [cplex.infinity] * self.n_dim
        self.Subproblem_QP.variables.add(obj = self.obj_linear, ub = ub, names = self.variablenames)
        self.Subproblem_QP.objective.set_quadratic(self.qmat)
    
        rownames = ['c' + str(i)  for i in range(self.n_constraints)]

        self.Subproblem_QP.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = rownames)
        self.Subproblem_QP.solve()
        self.get_solvable_status()
        self.get_optvalues()

    def write_model(self):
        self.Subproblem_QP.write('Subproblem_QP.lp')
        
    def get_optvalues(self):
        for i, x in enumerate(self.Subproblem_QP.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.Subproblem_QP.solution.get_status()
        if status == self.Subproblem_QP.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.Subproblem_QP.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.Subproblem_QP.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def output_QP_result(self):
        with open('Subproblem_QP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
            f.write('\n')

In [357]:
n_stage, pcast, n_batch, n_machine_perstage = 3, 5, 3, 3
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']

processtime = [14 * random.random() + 36 for i in range(n_job * n_stage)]
transittime = dict(zip([stage_name[i] + 'to' + stage_name[i+1] for i in range(n_stage - 1)], [3 * random.random() + 3 for i in range(n_stage - 1)]))
su = 80
duedate = [0 for i in range(pcast * n_machine['casting'])]

penalty_coeff = 10
d_coeff = {'d1': 10, 'd2': 110}
coeff = [10 + 20 * i for i in range(n_stage - 1)]

assert(len(stage_name) == n_stage)
print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of machine per stage:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)
print('transfer time:', transittime)

n_position = dict(zip(stage_name[0:n_stage - 1], [n_job / n_machine['steelmaking'] ] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

Number of stage: 3
Cast number of per machine: 5
Number of machine per stage: 3
Number of machine: {'steelmaking': 3, 'refining': 3, 'casting': 3}
Number of job: 45
transfer time: {'steelmakingtorefining': 3.333538671583243, 'refiningtocasting': 3.572715656196481}
n_position =  {'steelmaking': 15.0, 'refining': 15.0}


In [358]:
subproblem_lp = Subproblem_LP()
subproblem_lp.compute_obj()
subproblem_lp.solve()
subproblem_lp.write_model()

subproblem_qp = Subproblem_QP()
subproblem_qp.obj_linear, subproblem_qp.qmat = subproblem_qp.compute_obj_linear(), subproblem_qp.compute_obj_qmat()
subproblem_qp.solve()
subproblem_qp.write_model()

the dimension of x: 4050 the number of constraints: 180
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (1.14 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =            87.685741
Iteration:    80   Dual objective     =            86.696899

Dual simplex solved model.

[97.79422350508113, 84.92436897954232, 87.27322228272797]
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 13530
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.27 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 165
  Integer space required    = 165
  Total non-zeros in factor = 13695
  Total FP ops to factor    = 1511015
Tried aggregator 1 time.
QP Presolve eliminated 164 rows and 